You can use this template for problems 1, 2 and 3

In [2]:
import numpy as np
import time
# Students will submit their files with their team-name.py
# Student have to use the Team as their parent class

Recent Version of the Code

In [3]:
class ROLLNUMBER_Q1:
  def __init__(self):
    self.num_arms = 6
    self.wickets = np.zeros(self.num_arms)
    self.occur = np.zeros(self.num_arms)
    self.prev_action = 0
    self.round = 0
    
  @staticmethod
  def kl_div(p, q):
    if q==0 and p==0:
        return 0
    if q==0 and p!=0:
        return pow(10, 10)
    if q==1 and p==1:
        return 0
    if q==1 and p!=1:
        return pow(10, 10)
    if p==0:
        return np.log(1/(1-q))
    if p==1:
        return np.log(1/q)
    
    return p*np.log(p/q) + (1-p)*np.log((1-p)/(1-q))

  def kl_conf(self, t, em, num_pulls, precision=1e-6, max_iter=50):
    n = 0
    lower_bound = em
    upper_bound = 1
    
    while n<max_iter and (upper_bound-lower_bound)>precision:
        q = (lower_bound+upper_bound)/2
        if self.kl_div(em, q) < np.log(1 + t*np.log(t)**2)/num_pulls:
            upper_bound = q
        else:
            lower_bound = q
        n+= 1
    
    return (lower_bound+upper_bound)/2


  def get_action(self,wicket,runs_scored):
    X = wicket
    R = runs_scored
    
    if self.round > 0: 
        self.wickets[self.prev_action] += X
        self.occur[self.prev_action] += 1
        
    if self.round < 6:
        action = self.round
    
    else:
        kl_ucb = np.array([self.kl_conf(self.round, 1-self.wickets[idx]/self.occur[idx], self.occur[idx]) for idx in range(self.num_arms)])
        action = np.argmax(kl_ucb)
    
    self.prev_action = action
    self.round += 1
    
    return action
    
    


In [4]:
class Environment:
  def __init__(self,num_balls,agent):
    self.num_balls = num_balls
    self.agent = agent
    self.__run_time = 0
    self.__total_runs = 0
    self.__total_wickets = 0
    self.__runs_scored = 0
    self.__start_time = 0
    self.__end_time = 0
    self.__regret_w = 0
    self.__regret_s = 0
    self.__wicket = 0
    self.__regret_rho = 0
    self.__p_out = np.array([0.001,0.01,0.02,0.03,0.1,0.3])
    self.__p_out = np.array([0.01,0.001,0.02,0.003,0.1,0.3]) # Test 1
    self.__p_run = np.array([1,0.9,0.85,0.8,0.75,0.7])
    self.__action_runs_map = np.array([0,1,2,3,4,6])
    self.__s = (1-self.__p_out)*self.__p_run*self.__action_runs_map
    self.__rho = self.__s/self.__p_out


  def __get_action(self):
    self.__start_time      = time. time()
    action          = self.agent.get_action(self.__wicket,self.__runs_scored)
    self.__end_time        = time. time()
    self.__run_time   = self.__run_time + self.__end_time - self.__start_time
    return action


  def __get_outcome(self, action):
    pout = self.__p_out[action]
    prun= self.__p_run[action]
    wicket = np.random.choice(2,1,p=[1-pout,pout])[0]
    runs = 0
    if(wicket==0):
      runs = self.__action_runs_map[action]*np.random.choice(2,1,p=[1-prun,prun])[0]
    return wicket, runs


  def innings(self):
    self.__total_runs = 0
    self.__total_wickets = 0
    self.__runs_scored = 0

    for ball in range(self.num_balls):
      action = self.__get_action()
      self.__wicket, self.__runs_scored   = self.__get_outcome(action)
      self.__total_runs     = self.__total_runs + self.__runs_scored
      self.__total_wickets  = self.__total_wickets + self.__wicket
      self.__regret_w       = self.__regret_w+ (self.__p_out[action]-np.min(self.__p_out))
      self.__regret_s       = self.__regret_s+ (np.max(self.__s) - self.__s[action])
      self.__regret_rho       = self.__regret_rho+ (np.max(self.__rho)-self.__rho[action])
    return self.__regret_w,self.__regret_s,self.__regret_rho, self.__total_runs, self.__total_wickets, self.__run_time


In [5]:
agent = ROLLNUMBER_Q1()
environment = Environment(100,agent)
regret_w,regret_s,reger_rho,total_runs,total_wickets,run_time = environment.innings()

In [84]:
print(regret_w,regret_s,reger_rho,total_runs,total_wickets,run_time)

0.7160000000000002 2064.467899999976 32348.999999999978 848 2 0.19564056396484375


In [85]:
agent.occur

array([ 33., 962.,   1.,   1.,   1.,   1.])